<a href="https://colab.research.google.com/github/rezwan578/Cervical_cancer_screening_with_computer_vision/blob/main/VGG16_classification_VIA_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from zipfile import ZipFile
import os,glob
import cv2
from tqdm.notebook import tqdm_notebook as tqdm
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense,MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from zipfile import ZipFile

In [ ]:
!unzip "/content/VIA_1.zip"

In [ ]:
!ls
%cd /content
!ls

In [ ]:


# Load positive samples
os.chdir('/content/Positive')
X_pos = []
y_pos = []
for i in tqdm(os.listdir()):
    img = cv2.imread(i)
    img = cv2.resize(img, (224, 224))
    X_pos.append(img)
    y_pos.append(1)  # Assuming positive class is represented by 1

# Load negative samples
os.chdir('/content/Negative')
X_neg = []
y_neg = []
for i in tqdm(os.listdir()):
    img = cv2.imread(i)
    img = cv2.resize(img, (224, 224))
    X_neg.append(img)
    y_neg.append(0)  # Assuming negative class is represented by 0

# Ensure equal number of samples for positive and negative classes
min_samples = min(len(X_pos), len(X_neg))
X = X_pos[:min_samples] + X_neg[:min_samples]
y = y_pos[:min_samples] + y_neg[:min_samples]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Label Encoding
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Convert labels to one-hot encoding
num_classes = len(set(y_train))  # Assuming all classes are present in y_train
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

# Convert lists to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)

# Print shapes
print("X_train Shape:", X_train.shape)
print("X_test Shape:", X_test.shape)
print("y_train Shape:", y_train.shape)
print("y_test Shape:", y_test.shape)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
for i in range(4):
    plt.subplot(1, 4, i+1)
    plt.imshow(X[i], cmap="gray")
    plt.axis('off')
plt.show()

In [ ]:
from keras.applications import vgg16


img_rows, img_cols = 224, 224


vgg = vgg16.VGG16(weights = 'imagenet',
                 include_top = False,
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers
# Layers are set to trainable as True by default
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
for (i,layer) in enumerate(vgg.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)
    def lw(bottom_model, num_classes):
      top_model = bottom_model.output
      top_model = GlobalAveragePooling2D()(top_model)
      top_model = Dense(1024,activation='relu')(top_model)
      top_model = Dense(1024,activation='relu')(top_model)
      top_model = Dense(512,activation='relu')(top_model)
      top_model = Dense(num_classes,activation='softmax')(top_model)
      return top_model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

from keras.models import Model


num_classes = 2

FC_Head = lw(vgg, num_classes)

model = Model(inputs = vgg.input, outputs = FC_Head)

print(model.summary())
from tensorflow.keras.models import Model
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), verbose=1, initial_epoch=0)




In [ ]:
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']


epochs = range(len(acc))

In [ ]:
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
# prompt: Write me code for finding out precision and recall for this model

import numpy as np
from sklearn.metrics import precision_score, recall_score

y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

print("Precision:", precision)
print("Recall:", recall)


In [ ]:
# prompt: write me a code for finding out the AUC of this model

from sklearn.metrics import roc_auc_score

# Get predicted probabilities for the test set
y_pred_prob = model.predict(X_test)

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_prob)

# Print the AUC
print("AUC:", auc)


In [ ]:
# prompt: write me a code for plotting confusion matrix of this model

import matplotlib.pyplot as plt
import seaborn as sns

# Get the class labels
class_labels = ['Negative', 'Positive']

# Create the confusion matrix plot
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)

# Add labels and title
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.title("Confusion Matrix")

# Show the plot
plt.show()


In [ ]:
# prompt: write me a code for plotting percentage confusion matrix of this model

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Get the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Calculate the percentage of each class in the confusion matrix
cm_percent = cm / cm.sum(axis=1)[:, np.newaxis]

# Create a new plot figure
plt.figure(figsize=(8, 6))

# Plot the percentage confusion matrix
sns.heatmap(cm_percent, annot=True, fmt=".2%", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)

# Add labels and title
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.title("Percentage Confusion Matrix")

# Show the plot
plt.show()


In [ ]:
import cv2
import numpy as np
from google.colab import output
score1 = 0
# Load the image
def prediction_test(image_path):
  global score1
#image_path = "/content/Negative/AAA1_rotated_1_jpg.rf.fd3ab75b9c31be68429d2b1b289f8264.jpg"  # Replace "path_to_your_image.jpg" with the actual path to your image
  image = cv2.imread(image_path)

# Check if the image is loaded successfully
  if image is not None:
    # Resize the image to match the expected input shape of your model
      resized_image = cv2.resize(image, (224, 224))

    # Preprocess the image (if necessary)
    # You may need to apply further preprocessing based on how your model was trained

    # Make predictions
      prediction = model.predict(np.expand_dims(resized_image, axis=0))

    # Get the predicted class
      predicted_class = np.argmax(prediction)

    # Print the predicted class
      #print("Predicted Class:", predicted_class)

      if predicted_class == 0:
        score1 += 1
      else:
        print("error")

  else:
      print("Failed to load the image. Please check the image path.")

import os

dir = "/content/Negative"

c = 0

for filename in os.listdir(dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(dir, filename)
        prediction_test(image_path)
        c = c + 1
        if c == 200:
            break

output.clear()
print(score1)



In [ ]:
# prompt: write me a code to draw a percentage confusion matrix about positive negative with precision 0.95 recall 0.91 f1-score 0.93

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Define the confusion matrix
cm = np.array([[180, 20], [18, 172]])

# Define the class labels
class_labels = ['Negative', 'Positive']

# Calculate the percentage of each class in the confusion matrix
cm_percent = cm / cm.sum(axis=1)[:, np.newaxis]

# Create a new plot figure
plt.figure(figsize=(8, 6))

# Plot the percentage confusion matrix
sns.heatmap(cm_percent, annot=True, fmt=".2%", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)

# Add labels and title
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.title("Percentage Confusion Matrix")

# Add precision, recall, and f1-score
plt.text(0.5, 0.3, "Precision: 0.95", ha='center', va='center', transform=plt.gca().transAxes)
plt.text(0.5, 0.6, "Recall: 0.91", ha='center', va='center', transform=plt.gca().transAxes)
plt.text(0.5, 0.9, "F1-score: 0.93", ha='center', va='center', transform=plt.gca().transAxes)

# Show the plot
plt.show()
